In [5]:
import pandas as pd
import numpy as np
import json
df = pd.read_csv("../data/rummagenexrummageo.csv")
df = df.drop_duplicates(subset='pmc_id', keep='first')
df = df.sort_values(by=["cosine_similarity", "p-value", "odds"], ascending=[True, True, False])
df.index = np.arange(1, len(df)+1)
df = df.head(1000)
df

/var/folders/92/gfh58db97nv7kkklt8263w9w0000gp/T/ipykernel_22512/3824191928.py:5: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/rummagenexrummageo.csv")


,Unnamed: 0,rummagene,rummageo,rummagene-desc,p-value,sidak,fdr,odds,n-overlap,rummagene-size,rummageo-size,overlaps,species,geo_gse,pmc_id,pmid,geo_pmid,geo_gsei
0,0,PMC7185143-aging-12-102991-s001..docx-0-Gene_name,GSE159694-3-vs-0-human up,co-expression analysis of all ArrayExpress dat...,0.0,0.0,0.0,64.730851,183,219.0,284,STIL;POLE2;TTK;CDCA2;RAD51;NCAPD2;DEPDC1B;PKMY...,human,GSE159694,PMC7185143,32275642,33264627,GSE159694
1,1,PMC9762333-ccr-21-3329_supplementary_tables_1-...,GSE159694-3-vs-0-human up,Click here for additional data file,0.0,0.0,0.0,63.068678,184,226.0,284,XRCC2;MCM3;SKA3;NCAPG;MELK;PCNA;NUF2;ZNF367;FA...,human,GSE159694,PMC9762333,34965943,33264627,GSE159694
2,2,PMC7185143-aging-12-102991-s001..docx-0-Gene_name,GSE184536-7-vs-11-human dn,co-expression analysis of all ArrayExpress dat...,0.0,0.0,0.0,62.962248,178,219.0,284,STIL;TTK;CDCA2;RAD51;NCAPD3;NCAPD2;DEPDC1B;PKM...,human,GSE184536,PMC7185143,32275642,34523966,GSE184536
3,3,PMC9762333-ccr-21-3329_supplementary_tables_1-...,GSE184536-7-vs-11-human dn,Click here for additional data file,0.0,0.0,0.0,62.040384,181,226.0,284,XRCC2;MCM3;SKA3;NCAPG;MELK;GINS4;PCNA;NUF2;ZNF...,human,GSE184536,PMC9762333,34965943,34523966,GSE184536
4,4,PMC7185143-aging-12-102991-s001..docx-0-Gene_name,GSE159694-3-vs-2-human up,co-expression analysis of all ArrayExpress dat...,0.0,0.0,0.0,51.160932,192,219.0,377,STIL;POLE2;TTK;CDCA2;RAD51;NCAPD3;NCAPD2;DEPDC...,human,GSE159694,PMC7185143,32275642,33264627,GSE159694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,PMC5794756-41598_2018_20217_MOESM4_ESM.xlsx-Do...,"GSE119336,GSE119337-0-vs-3-human up",were upregulated and 835 were downregulated in...,0.0,0.0,0.0,13.020687,378,628.0,1017,CNIH1;F9;NR1I3;C6;SFXN1;TLCD4;CD36;KLKB1;IGFBP...,human,GSE119336,PMC5794756,29391527,36476255,"GSE119336,GSE119337"
996,996,PMC4951018-pone.0159329.s004.xlsx-Cluster_2_67...,GSE166238-2-vs-6-human dn,Expression values of all upregulated genes. (c...,0.0,0.0,0.0,13.018896,397,589.0,1139,BMAL2;NBN;ICAM1;IFIT2;PARP9;BATF2;HIVEP2;PELI1...,human,GSE166238,PMC4951018,27434537,"34267957,35930640",GSE166238
997,997,PMC9362915-pone.0272017.s004.xlsx-Day0_0_0_Top...,GSE210352-5-vs-6-mouse dn,them. A full list of genes for each path is pr...,0.0,0.0,NaN,13.015083,479,714.0,1134,PALM;PPM1J;E2F3;NCAPH;EGFL7;MAP3K8;CCNE2;S1PR5...,mouse,GSE210352,PMC9362915,35944008,Empty,GSE210352
998,998,PMC11188533-12993_2024_240_MOESM2_ESM.xlsx-Fig...,GSE75246-4-vs-6-mouse up,None here,0.0,0.0,0.0,13.011917,351,464.0,1279,SORBS1;P2RY14;MAP2;ASTN2;PDZRN4;GRIN2A;CHSY3;E...,mouse,GSE75246,PMC11188533,38898502,27097852,GSE75246


In [7]:
df_rummagene_map = {row["rummagene"]: [row["rummagene-desc"], row["pmc_id"]] for _, row in df.iterrows()}
df_rummageo_map = {row["rummageo"]: [row["geo_gse"], row["geo_gsei"], row["species"]] for _, row in df.iterrows()}


In [10]:
with open("../data/gse_results.json" ,"r") as f:
        gse_info = json.load(f)
with open("../data/title_abs.json" ,"r") as f:
        pmc_sum = json.load(f)
with open("../data/human-gse-processed-meta.json") as f:
        gse_human_info = json.load(f)
with open("../data/mouse-gse-processed-meta.json") as f:
        gse_mouse_info = json.load(f)

In [11]:
import re

def extract_after_extension(termid):
    pattern = r'^(.+?\.\w+)-+(.*)$'
    
    match = re.match(pattern, termid)
    
    if match:
        return match.group(2)
    else:
        return None


In [13]:
def pmc_again(pmcids):
    pmc_dict = {}
    for pm in pmcids:
        pmc_dict[pm] = (pmc_sum[pmcids[pm][1]]["title"], pmc_sum[pmcids[pm][1]]["abstract"], extract_after_extension(pm), pmcids[pm][0])

    return pmc_dict


fin_pmc = pmc_again(df_rummagene_map)
fin_pmc


{'PMC7185143-aging-12-102991-s001..docx-0-Gene_name': ('PCNA-associated factor KIAA0101 transcriptionally induced by ELK1 controls cell proliferation and apoptosis in nasopharyngeal carcinoma: an integrated bioinformatics and experimental study',
  'KIAA0101, previously identified as PCNA-associated factor, is overexpressed among almost majority of human cancers and has emerged as an important regulator of cancer progression; however, its function in human nasopharyngeal carcinoma (NPC) remain unknown. Integrated bioinformatics approaches were employed to determine the KIAA0101 expressions in the NPC samples. Lentiviral vectors carrying KIAA0101 shRNA were constructed and stable transfected cells were validated by qRT-PCR and western blot. Cellular functions were then evaluated by MTT, colony formation, Brdu staining, and flow cytometry. Mechanistic studies were systematically investigated by UCSC Genome Browser, GEO, UALCAN, QIAGEN, PROMO and JASPAR, ChIP, and the cBioPortal, et al. T

In [28]:
def gse_again(gseids):
    gse_dict = {}
    for gse in gseids:
        if gseids[gse][2] == "human":
            dt = gse_human_info
        else:
            dt = gse_mouse_info
        title = gse_info[gseids[gse][0]]["title"] 
        summary = gse_info[gseids[gse][0]]["summary"][0]
        cond1= dt[gseids[gse][1]]["titles"][gse.split("-")[1]] 
        cond2 = dt[gseids[gse][1]]["titles"][gse.split("-")[3]]
        dir = gse.split("-")[4].split(" ")[0]
        gse_dict[gse] = (title, summary, cond1,cond2, dir)
    return gse_dict

fin_gse = gse_again(df_rummageo_map)
fin_gse


{'GSE159694-3-vs-0-human up': (['Retinoic Acid Promotes Endothelial Cell Cycle Early G1 State to Enable Human Hemogenic Endothelial Cell Specification'],
  'Endothelial cells derived from hESCs were separated into 4 different cell cycle phases via FACS and sequenced (i.e., early G1, late G1, G1/S, S/G2/M)',
  'endothelial cells late g1 phase rep hesc derived untreated',
  'endothelial cells g1/ phase rep hesc derived untreated',
  'human'),
 'GSE184536-7-vs-11-human dn': (['The NF-κB transcriptional footprint is essential for SARS-CoV-2 replication'],
  'SARS-CoV-2, the etiological agent of COVID-19, is characterized by a delay in Type I interferon (IFN-I)-mediated antiviral defenses alongside robust cytokine production. Here we investigate the underlying molecular basis for this imbalance and implicate virus-mediated activation of NF-κB in the absence of other canonical IFN-I-related transcription factors. Epigenetic and single cell transcriptomic analyses show a selective NF-κB signa

In [ ]:
import requests
import time

def get_enrichr_terms(genes):
    ENRICHR_URL = 'https://maayanlab.cloud/Enrichr/'
    endpoint = 'addList'
    user_list_id = None
    enrichr_libraries = ['WikiPathway_2023_Human', 'GWAS_Catalog_2023', 
                         'GO_Biological_Process_2023', 'MGI_Mammalian_Phenotype_Level_4_2021']
    
    
    try:
        gen = genes.split(";")
        genes_string = '\n'.join(gen).replace("'", '')
        response = requests.post(
            ENRICHR_URL + endpoint,
            files={'list': (None, genes_string), 'description': (None, '')}
            # headers={'Content-Type': 'multipart/form-data'}
        )
        response.raise_for_status()
        data = response.json()
        user_list_id = data['userListId']
    except requests.exceptions.RequestException as error:
        print(f"Error: {error}")
        return

    enriched_terms = {}
    enrichr_stats = {}

    for enrichr_library in enrichr_libraries:
        query_string = f'enrich?userListId={user_list_id}&backgroundType={enrichr_library}'
        try:
            response = requests.get(ENRICHR_URL + query_string, headers={'Accept': 'application/json'})
            response.raise_for_status()
            data = response.json()

            enriched_terms[enrichr_library] = []
            for term in data[enrichr_library][:3]:  # Limit to top 3 results
                term_name = term[1]
                enriched_terms[enrichr_library].append(term_name)
                enrichr_stats[term_name] = term
                enrichr_stats[term_name].append(enrichr_library)

            time.sleep(0.5)  # Delay between requests

        except requests.exceptions.RequestException as error:
            print(f"Error: {error}")
            return

    return enriched_terms, enrichr_stats


In [ ]:
# import requests
# import os
# import json

# def fetch_hypothesis(pm_title, user_desc: str, desct, desc, gse_title, gse_summary: str, cond1, cond2, species, term1: str,term2: str, enriched_terms: dict, enriched_stats: dict) -> str:
#     # Define the system prompt
#     system_prompt = (
#         "You are an AI hypothesis generator for RummagenexRummaGEO (gene sets from crossing Rummagene sets with RummaGEO sets). "
#         "You should act as a biologist in hypothesizing why a high overlap may exist between a Rummagene set (which are sets from PubMed papers) and a RummaGEO set (automatically generated signatures from Gene Expression Omnibus)."
#     )

#     # Build the enriched terms string
#     enriched_terms_string = ""
#     for library, terms in enriched_terms.items():
#         enriched_terms_string += f"{library}: {', '.join(terms)}\n"

#     # Define the main prompt to be sent to the AI
#     prompt = (
#         f"Here are two gene sets that highly overlap. The first is from a Rummagene set. "
#         f"The second is a gene set automatically computed between two conditions in a study from the Gene Expression Omnibus (GEO). "
#         f"Based upon the term name (formatted as condition 1 vs. condition 2) and the abstract of the GEO gene set, "
#         f"and the abstract of the Rummagene gene set, please hypothesize about why these two gene sets have a significant high overlap. "
#         f"You should mention both the summary of the RummaGEO gene set and the description of the Rummagene gene set in your hypothesis. "
#         f"You will also be provided with enrichment results from the Enrichr database to help you generate your hypothesis, which shows "
#         f"significantly overlapping functional terms from the overlapping genes of the two sets. "
#         f"For each enrichment term that appears in your response, the term should appear in the exact form it was given to you "
#         f"(do not exclude any words or characters from a term. For example, Complement And Coagulation Cascades WP558 should appear as "
#         f"Complement And Coagulation Cascades WP558, not Complement And Coagulation Cascades). Also, please don't use quotes around the enriched term names. "
#         f"Gene set term 1 from RummaGEO: {term2}\n"
#         f'"up" or "dn" in this{term2} name indicates if the genes were upregulated or downregulated in {cond1} vs {cond2} conditions in the signature for species {species}.Please make sure to include this detail in your hypothesis.\n'
#         f"title of study for gene set term 1: {gse_title}\n"
#         f"summary of study for gene set term 1: {gse_summary}\n"
#         f"Gene set term 2 from Rummagene: {term1}\n"
#         f"title of paper for gene set term 2: {pm_title}\n"
#         f"abstract of paper for gene set term 2: {user_desc}\n"
#         f"{desct}is the name of the table from the paper which the gene set comes from and {desc} is its description. Please include this detail if relevant\n"
#         f"Enriched Terms from overlapping genes of the two sets:\n"
#         f"{enriched_terms_string}"
#     )

#     # Send the request to the OpenAI API
#     response = requests.post(
#         "https://api.openai.com/v1/chat/completions",
#         headers={
#             "Content-Type": "application/json",
#             "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}",
#         },
#         json={
#             "model": "gpt-4o",
#             "messages": [
#                 {"role": "system", "content": system_prompt},
#                 {"role": "user", "content": prompt}
#             ],
#             "max_tokens": 1000,
#             "temperature": 0
#         }
#     )

#     # Parse the response from the API
#     response_json = response.json()
#     hypothesis = response_json['choices'][0]['message']['content']

#     # Replace enriched terms in the hypothesis with detailed text
#     for term, stats in enriched_stats.items():
#         if term in hypothesis:
#             details = (
#                 f"Term: {term}\tLibrary: {stats[9]}\tRank: {stats[0]}\tP-value: {float(stats[2]):.2e}\tOdds Ratio: {float(stats[3]):.4f}\n"
#             )
#             hypothesis = hypothesis.replace(term, f"{term} ({details})")

#     return hypothesis


In [ ]:
import os
import requests
import json
from google.auth import jwt
from google.auth.transport.requests import Request

def fetch_hypothesis(pm_title, user_desc: str, desct, desc, gse_title, gse_summary: str, cond1, cond2, species, term1: str, term2: str, enriched_terms: dict, enriched_stats: dict) -> str:
    # Define the system prompt
    system_prompt = (
        "You are an AI hypothesis generator for RummagenexRummaGEO (gene sets from crossing Rummagene sets with RummaGEO sets). "
        "You should act as a biologist in hypothesizing why a high overlap may exist between a Rummagene set (which are sets from PubMed papers) and a RummaGEO set (automatically generated signatures from Gene Expression Omnibus)."
    )

    # Build the enriched terms string
    enriched_terms_string = ""
    for library, terms in enriched_terms.items():
        enriched_terms_string += f"{library}: {', '.join(terms)}\n"

    # Define the main prompt to be sent to the AI
    prompt = (
        f"Here are two gene sets that highly overlap. The first is from a Rummagene set. "
        f"The second is a gene set automatically computed between two conditions in a study from the Gene Expression Omnibus (GEO). "
        f"Based upon the term name (formatted as condition 1 vs. condition 2) and the abstract of the GEO gene set, "
        f"and the abstract of the Rummagene gene set, please hypothesize about why these two gene sets have a significant high overlap. "
        f"You should mention both the summary of the RummaGEO gene set and the description of the Rummagene gene set in your hypothesis. "
        f"You will also be provided with enrichment results from the Enrichr database to help you generate your hypothesis, which shows "
        f"significantly overlapping functional terms from the overlapping genes of the two sets. "
        f"For each enrichment term that appears in your response, the term should appear in the exact form it was given to you "
        f"(do not exclude any words or characters from a term. For example, Complement And Coagulation Cascades WP558 should appear as "
        f"Complement And Coagulation Cascades WP558, not Complement And Coagulation Cascades). Also, please don't use quotes around the enriched term names. "
        f"Gene set term 1 from RummaGEO: {term2}\n"
        f'"up" or "dn" in this{term2} name indicates if the genes were upregulated or downregulated in {cond1} vs {cond2} conditions in the signature for species {species}.Please make sure to include this detail in your hypothesis.\n'
        f"title of study for gene set term 1: {gse_title}\n"
        f"summary of study for gene set term 1: {gse_summary}\n"
        f"Gene set term 2 from Rummagene: {term1}\n"
        f"title of paper for gene set term 2: {pm_title}\n"
        f"abstract of paper for gene set term 2: {user_desc}\n"
        f"{desct} is the name of the table from the paper which the gene set comes from and {desc} is its description. Please include this detail if relevant\n"
        f"Enriched Terms from overlapping genes of the two sets:\n"
        f"{enriched_terms_string}"
    )

    # Authenticate with the Gemini API
    credentials = jwt.Credentials.from_service_account_file(
        "path/to/service_account.json",
        audience="https://gemini.googleapis.com",
    )
    auth_request = Request()
    credentials.refresh(auth_request)
    
    # Send the request to the Gemini API
    response = requests.post(
        "https://gemini.googleapis.com/v1/models/gemini-model/complete",
        headers={
            "Content-Type": "application/json",
            "Authorization": f"Bearer {credentials.token}",
        },
        json={
            "model": "gemini-model",
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            "max_tokens": 1000,
            "temperature": 0
        }
    )

    # Parse the response from the API
    response_json = response.json()
    hypothesis = response_json.get('choices', [{}])[0].get('message', {}).get('content', '')

    # Replace enriched terms in the hypothesis with detailed text
    for term, stats in enriched_stats.items():
        if term in hypothesis:
            details = (
                f"Term: {term}\tLibrary: {stats[9]}\tRank: {stats[0]}\tP-value: {float(stats[2]):.2e}\tOdds Ratio: {float(stats[3]):.4f}\n"
            )
            hypothesis = hypothesis.replace(term, f"{term} ({details})")

    return hypothesis


In [ ]:
tuples_list = list(zip(df["rummagene"], df['rummageo'], df["overlaps"]))
hypotheses = {}


In [ ]:
for i, (pmcid, geo_id, ov) in enumerate(tuples_list):
        term = f"{pmcid};{geo_id}"
        if term not in hypotheses: 
            try:
                terms, stats = get_enrichr_terms(ov)
                hypothesis = fetch_hypothesis(pm_title=fin_pmc[pmcid][0], user_desc=fin_pmc[pmcid][1], desct=fin_pmc[pmcid][2], desc=fin_pmc[pmcid][3], gse_title=fin_gse[geo_id][0],gse_summary=fin_gse[geo_id][1], cond1=fin_gse[geo_id][2], cond2=fin_gse[geo_id][3],species=fin_gse[geo_id][4], term1=pmcid, term2=geo_id, enriched_terms=terms, enriched_stats=stats)
                hypotheses[term] = hypothesis
                with open('data_hyp.json', 'w') as json_file:
                    json.dump(hypotheses, json_file, indent=4)
            except:
                print(term)
                print(i)

   
